In [1]:
import mediapipe as mp
import cv2
import os
import numpy as np
from tqdm import tqdm   # progress bar

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

DATA_DIR = "data"
OUTPUT_DIR = "landmarks"
os.makedirs(OUTPUT_DIR, exist_ok=True)

categories = ['1','2','3','4','5','6','7','8','9'] + \
             [chr(i) for i in range(ord('A'), ord('Z')+1)]

X, y = [], []

for label, category in enumerate(categories):
    folder = os.path.join(DATA_DIR, category)
    if not os.path.exists(folder):
        print(f"⚠️ Skipping missing folder: {folder}")
        continue

    print(f"Processing category: {category}")
    for img_file in tqdm(os.listdir(folder), desc=f"{category}"):
        img_path = os.path.join(folder, img_file)
        img = cv2.imread(img_path)

        if img is None:
            print(f"⚠️ Could not read image {img_path}")
            continue

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = hands.process(img_rgb)

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                # Extract (x, y, z) for each of the 21 landmarks
                landmarks = []
                for lm in hand_landmarks.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])  
                X.append(landmarks)
                y.append(label)

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Save dataset
np.save(os.path.join(OUTPUT_DIR, "X.npy"), X)
np.save(os.path.join(OUTPUT_DIR, "y.npy"), y)

print(f"✅ Saved landmark dataset: {X.shape[0]} samples, {X.shape[1]} features each")


Processing category: 1


1: 100%|██████████| 1200/1200 [02:43<00:00,  7.33it/s]


Processing category: 2


2: 100%|██████████| 1200/1200 [03:07<00:00,  6.39it/s]


Processing category: 3


...

Processing category: Y


Y: 100%|██████████| 1200/1200 [01:09<00:00, 17.36it/s]


Processing category: Z


Z: 100%|██████████| 1200/1200 [01:09<00:00, 17.24it/s]


✅ Saved landmark dataset: 41143 samples, 63 features each


In [2]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# =========================
# Load Data
# =========================
X = np.load("landmarks/X.npy")
Y = np.load("landmarks/Y.npy")

print("Dataset:", X.shape, Y.shape)

# Encode labels
le = LabelEncoder()
Y_encoded = le.fit_transform(Y)
Y_cat = to_categorical(Y_encoded)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, Y_cat, test_size=0.2, random_state=42, stratify=Y_cat)

# =========================
# Build Model
# =========================
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(len(le.classes_), activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# =========================
# Train
# =========================
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    verbose=1
)

# =========================
# Evaluate
# =========================
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"✅ Test Accuracy: {acc*100:.2f}%")

# Save model + encoder
model.save("sign_language_model.h5")
import pickle
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)


Dataset: (41143, 63) (41143,)


c:\Users\lenovo\anaconda3\envs\myenv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 35)             │         2,275 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,723 (73.14 KB)

 Trainable params: 18,723 (73.14 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
1029/1029 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.3518 - loss: 2.3588 - val_accuracy: 0.9536 - val_loss: 0.3047
Epoch 2/30
1029/1029 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8588 - loss: 0.4844 - val_accuracy: 0.9881 - val_loss: 0.1014
Epoch 3/30
1029/1029 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9268 - loss: 0.2562 - val_accuracy: 0.9926 - val_loss: 0.0529
Epoch 4/30
1029/1029 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9546 - loss: 0.1689 - val_accuracy: 0.9951 - val_loss: 0.0293
Epoch 5/30
1029/1029 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9655 - loss: 0.1238 - val_accuracy: 0.9951 - val_loss: 0.0177
Epoch 6/30
1029/1029 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9738 - loss: 0.0937 - val_accuracy: 0.9957 - val_loss: 0.0132
Epoch 7/30
1029/1029 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9772 - loss: 0.0837 - val_accuracy: 0.9954 - val_loss: 0.0124
Epoch 8/30
1029/1029 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9789 - loss: 0.0698 - 

✅ Test Accuracy: 99.95%


In [1]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf

print("OpenCV version:", cv2.__version__)
print("NumPy version:", np.__version__)
print("Mediapipe version:", mp.__version__)
print("TensorFlow version:", tf.__version__)

OpenCV version: 4.11.0
NumPy version: 1.26.4
Mediapipe version: 0.10.21
TensorFlow version: 2.18.0
